In [149]:
# full_scanvi_pipeline.py
import os
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.metrics import adjusted_rand_score, v_measure_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import scvi
from scvi.model import SCANVI

In [150]:
random_seed = 42
np.random.seed(random_seed)
sc.settings.seed = random_seed

# Step 0: Download the data

In [151]:
bulk_train_x = pd.read_csv("train_data/train_bulk.csv", index_col=0)
bulk_train_y = pd.read_csv("train_data/train_bulk_trueprops.csv", index_col=0)
bulk_test_x = pd.read_csv("test_data/test_bulk.csv", index_col=0)

In [152]:
sc_train = sc.read_h5ad("train_data/train_adata.h5ad")
sc_test = sc.read_h5ad("test_data/test_adata.h5ad")

In [153]:
print(sc_train.var.shape) # gene level
sc_train.var.head()

(7725, 6)


,n_cells,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts
Unnamed: 0,,,,,,
NOC2L,6735,False,6735,0.143681,87.143757,7527.0
HES4,8287,False,8287,0.330635,84.181190,17321.0
ISG15,19640,False,19640,0.871934,62.509783,45678.0
TNFRSF18,12021,False,12021,0.682345,77.053467,35746.0
TNFRSF4,7880,False,7880,0.484510,84.958100,25382.0


In [154]:
print(sc_train.obs.shape) # cell level
sc_train.obs.head()

(32374, 10)


,Sample,Patient,Tumor status,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,highLevelType,chemo
AAACCCAAGGAGACCT-1_1,s1,P1,Tumor,991,991,2156.0,131.0,6.076067,T,post
AAACCCAGTAGACAGC-1_1,s1,P1,Tumor,1063,1063,2485.0,84.0,3.380282,T,post
AAACCCAGTAGATCGG-1_1,s1,P1,Tumor,855,855,1993.0,87.0,4.365278,T,post
AAACCCAGTATCACCA-1_1,s1,P1,Tumor,1002,1002,2147.0,139.0,6.474150,B,post
AAACCCAGTTGGAGGT-1_1,s1,P1,Tumor,958,958,2438.0,108.0,4.429861,T,post


In [155]:
print(sc_train.X.shape) # expression of the cells by genes

(32374, 7725)


# Step 1: Perform SC clustering 

In [156]:
sc.pp.filter_cells(sc_train, min_genes=200)
sc.pp.filter_genes(sc_train, min_cells=3)
sc_train = sc_train[sc_train.obs["pct_counts_mt"] < 10]

In [157]:
#should i cut it in both and then assigned the netirety to the clusters
sc.pp.filter_genes(sc_test, min_cells=3)

In [158]:
# Match genes
common_genes = sc_train.var_names.intersection(sc_test.var_names)
sc_train = sc_train[:, common_genes].copy()
sc_test  = sc_test[:, common_genes].copy()


In [159]:
label_key = "highLevelType"
batch_key = "Sample"

In [ ]:
# Set ALL test labels to Unknown
sc_test.obs[label_key] = "Unknown"

In [ ]:
combined = sc_train.concatenate(sc_test, batch_key="dataset", batch_categories=["train","test"], index_unique=None)
combined.obs[label_key] = combined.obs[label_key].astype("category")

C:\Users\edonn\AppData\Local\Temp\ipykernel_14660\2565407024.py:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  combined = sc_train.concatenate(sc_test, batch_key="dataset", batch_categories=["train","test"], index_unique=None)


In [164]:
# Setup AnnData for scANVI
scvi.model.SCANVI.setup_anndata(combined, batch_key=batch_key, labels_key=label_key, unlabeled_category="Unknown")

# Choose n_latent (e.g., 20-50).
scanvi_model = scvi.model.SCANVI(combined, n_latent=30)
scanvi_model.train(max_epochs=200, early_stopping=True, check_val_every_n_epoch=10)

INFO     Training for 200 epochs.                                                                                  


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
c:\Users\edonn\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\edonn\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\edonn\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `Data

Epoch 200/200: 100%|██████████| 200/200 [2:24:44<00:00, 44.13s/it, v_num=1, train_loss=3.82e+3]  

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [2:24:44<00:00, 43.42s/it, v_num=1, train_loss=3.82e+3]



In [165]:
# Extract latent representation for combined and split back into train/test
Z = scanvi_model.get_latent_representation()
n_train = sc_train.n_obs
Z_train = Z[:n_train, :]
Z_test  = Z[n_train:, :]

# Attach latent to original objects (so downstream Scanpy neighbors/leiden work)
sc_train.obsm["X_scANVI"] = Z_train
sc_test.obsm["X_scANVI"]  = Z_test

In [166]:
# Build neighbors and tune resolution on TRAIN latent
sc.pp.neighbors(sc_train, use_rep="X_scANVI", n_neighbors=15)
sc.tl.umap(sc_train, min_dist=0.3)

In [167]:
# resolution search helper
def search_best_resolution(adata, true_label_key="highLevelType", nsteps=30, low=0.2, high=1.5):
    true_labels = adata.obs[true_label_key].values
    best_score = -1.0
    best_res = None
    for r in np.linspace(low, high, nsteps):
        tmp = adata.copy()
        sc.tl.leiden(tmp, resolution=float(r), key_added="tmp_cluster")
        pred = tmp.obs["tmp_cluster"].astype(str).values
        ari = adjusted_rand_score(true_labels, pred)
        v = v_measure_score(true_labels, pred)
        score = ari + v
        if score > best_score:
            best_score = score
            best_res = float(r)
    return best_res, best_score

best_res, best_score = search_best_resolution(sc_train, true_label_key="highLevelType", nsteps=30, low=0.2, high=1.2)
print("Best resolution chosen (train):", best_res, "score:", best_score)

Best resolution chosen (train): 0.23448275862068968 score: 1.3303885159728608


In [168]:

# Final clustering on train
sc.tl.leiden(sc_train, resolution=best_res, key_added="cluster")

# Step 2: Deconvolve the data

In [ ]:
# for each cell type x, cacluate x_gene_expression = x_cell_percentage * sum(over all clusters assigned to cell type x) (cluster_cell_percentage * cluster_expression_matrix)

# Step 2bis: Predict on the test data

In [169]:
# Assign clusters for test via KNN in latent space
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(sc_train.obsm["X_scANVI"], sc_train.obs["cluster"])
sc_test.obs["cluster"] = knn.predict(sc_test.obsm["X_scANVI"])

In [171]:
# Optionally save predicted labels (with cell barcodes in index)
df_test = sc_test.obs[["cluster"]].reset_index().rename(columns={"index": "cell_id"})

In [173]:
results = sc_test.obs["cluster"]
results = results.reset_index()

In [ ]:
pd.DataFrame(results).to_csv("results/cluster_membership.csv")

# Step 3: Save the required files

In [ ]:
# pred_props should be a DataFrame containing the estimated cell type proportions for the patients in all_bulkified
# pred_props.columns = ['index','s5_0','s5_1',...,'s10_3','s10_4'] = np.append(["index"],all_bulkified.columns)
# pred_props['index'] = ['T', 'Endothelial', 'Fibroblast', 'Plasmablast', 'B', 'Myofibroblast',
#       'NK', 'Myeloid', 'Mast']

In [175]:
results_path = pl.Path("results")

In [ ]:
# cluster_labels should be a DataFrame containing the cluster labels for each cell
# cluster_labels.columns = ["index", "cluster"]
# cluster_labels["index"] = test_adata.columns

In [ ]:
pred_props = pd.read_csv("results/pred_props.csv", index_col=0)
cluster_labels = pd.read_csv("results/cluster_membership.csv", index_col=0)

In [ ]:
import zipfile

archive_name = "Negri_Edoardo_Project2.zip" # TODO

with zipfile.ZipFile(results_path / archive_name, "x") as zf:
    with zf.open(f"pred_props.csv", "w") as buffer:
        pred_props.to_csv(buffer)
    with zf.open(f"cluster_membership.csv", "w") as buffer:
        cluster_labels.to_csv(buffer)
    zf.close()